<a href="https://colab.research.google.com/github/doukansurel/Retrieval-Augmented-Generation/blob/main/OpenAI_Agent_Query_PlanningwithLlamaIndex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index

In [2]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 3.6 MB/s eta 0:00:00


In [4]:
import os
from google.colab import drive

os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
from llama_index import (
    SimpleDirectoryReader,
    ServiceContext,
    GPTVectorStoreIndex,
)
from llama_index.response.pprint_utils import pprint_response
from llama_index.llms import OpenAI

llm = OpenAI(temperature=0, model="gpt-4")
service_context = ServiceContext.from_defaults(llm=llm)

In [ ]:
!mkdir -p 'data/10q/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10q/uber_10q_march_2022.pdf' -O 'data/10q/uber_10q_march_2022.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10q/uber_10q_june_2022.pdf' -O 'data/10q/uber_10q_june_2022.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10q/uber_10q_sept_2022.pdf' -O 'data/10q/uber_10q_sept_2022.pdf'

In [8]:
march_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_march_2022.pdf"]
).load_data()
june_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_june_2022.pdf"]
).load_data()
sept_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_sept_2022.pdf"]
).load_data()

In [9]:
march_index = GPTVectorStoreIndex.from_documents(march_2022)
june_index = GPTVectorStoreIndex.from_documents(june_2022)
sept_index = GPTVectorStoreIndex.from_documents(sept_2022)

In [10]:
march_engine = march_index.as_query_engine(
    similarity_top_k=3, service_context=service_context
)
june_engine = june_index.as_query_engine(
    similarity_top_k=3, service_context=service_context
)
sept_engine = sept_index.as_query_engine(
    similarity_top_k=3, service_context=service_context
)

In [11]:
from llama_index.tools import QueryEngineTool


query_tool_sept = QueryEngineTool.from_defaults(
    query_engine=sept_engine,
    name="sept_2022",
    description=(
        f"Provides information about Uber quarterly financials ending"
        f" September 2022"
    ),
)
query_tool_june = QueryEngineTool.from_defaults(
    query_engine=june_engine,
    name="june_2022",
    description=(
        f"Provides information about Uber quarterly financials ending June"
        f" 2022"
    ),
)
query_tool_march = QueryEngineTool.from_defaults(
    query_engine=march_engine,
    name="march_2022",
    description=(
        f"Provides information about Uber quarterly financials ending March"
        f" 2022"
    ),
)

In [12]:
# define query plan tool
from llama_index.tools import QueryPlanTool
from llama_index import get_response_synthesizer

response_synthesizer = get_response_synthesizer(
    service_context=service_context
)
query_plan_tool = QueryPlanTool.from_defaults(
    query_engine_tools=[query_tool_sept, query_tool_june, query_tool_march],
    response_synthesizer=response_synthesizer,
)

In [13]:
query_plan_tool.metadata.to_openai_tool()

{'type': 'function',
 'function': {'name': 'query_plan_tool',
  'description': '        This is a query plan tool that takes in a list of tools and executes a query plan over these tools to answer a query. The query plan is a DAG of query nodes.\n\nGiven a list of tool names and the query plan schema, you can choose to generate a query plan to answer a question.\n\nThe tool names and descriptions are as follows:\n\n\n\n        Tool Name: sept_2022\nTool Description: Provides information about Uber quarterly financials ending September 2022 \n\nTool Name: june_2022\nTool Description: Provides information about Uber quarterly financials ending June 2022 \n\nTool Name: march_2022\nTool Description: Provides information about Uber quarterly financials ending March 2022 \n        ',
  'parameters': {'title': 'QueryPlan',
   'description': "Query plan.\n\nContains a list of QueryNode objects (which is a recursive object).\nOut of the list of QueryNode objects, one of them must be the root no

In [14]:
from llama_index.agent import OpenAIAgent
from llama_index.llms import OpenAI


agent = OpenAIAgent.from_tools(
    [query_plan_tool],
    max_function_calls=10,
    llm=OpenAI(temperature=0, model="gpt-4-0613"),
    verbose=True,
)

In [15]:
response = agent.query("What were the risk factors in sept 2022?")


Added user message to memory: What were the risk factors in sept 2022?
=== Calling Function ===
Calling function: query_plan_tool with args: {
  "nodes": [
    {
      "id": 1,
      "query_str": "What were the risk factors in sept 2022?",
      "tool_name": "sept_2022",
      "dependencies": []
    }
  ]
}
Executing node {"id": 1, "query_str": "What were the risk factors in sept 2022?", "tool_name": "sept_2022", "dependencies": []}
Selected Tool: ToolMetadata(description='Provides information about Uber quarterly financials ending September 2022', name='sept_2022', fn_schema=<class 'llama_index.tools.types.DefaultToolFnSchema'>)
Executed query, got response.
Query: What were the risk factors in sept 2022?
Response: The risk factors in September 2022 included failure to meet regulatory requirements related to climate change or to meet stated climate change commitments, which could impact costs, operations, brand, and reputation. Outbreaks of contagious diseases like the COVID-19 pandem

In [16]:
from llama_index.tools.query_plan import QueryPlan, QueryNode

query_plan = QueryPlan(
    nodes=[
        QueryNode(
            id=1,
            query_str="risk factors",
            tool_name="sept_2022",
            dependencies=[],
        )
    ]
)

In [17]:
QueryPlan.schema()

{'title': 'QueryPlan',
 'description': "Query plan.\n\nContains a list of QueryNode objects (which is a recursive object).\nOut of the list of QueryNode objects, one of them must be the root node.\nThe root node is the one that isn't a dependency of any other node.",
 'type': 'object',
 'properties': {'nodes': {'title': 'Nodes',
   'description': 'The original question we are asking.',
   'type': 'array',
   'items': {'$ref': '#/definitions/QueryNode'}}},
 'required': ['nodes'],
 'definitions': {'QueryNode': {'title': 'QueryNode',
   'description': 'Query node.\n\nA query node represents a query (query_str) that must be answered.\nIt can either be answered by a tool (tool_name), or by a list of child nodes\n(child_nodes).\nThe tool_name and child_nodes fields are mutually exclusive.',
   'type': 'object',
   'properties': {'id': {'title': 'Id',
     'description': 'ID of the query node.',
     'type': 'integer'},
    'query_str': {'title': 'Query Str',
     'description': 'Question we 

In [18]:
response = agent.query(
    "Analyze Uber revenue growth in March, June, and September"
)

Added user message to memory: Analyze Uber revenue growth in March, June, and September
=== Calling Function ===
Calling function: query_plan_tool with args: {
  "nodes": [
    {
      "id": 1,
      "query_str": "What is Uber's revenue for March 2022?",
      "tool_name": "march_2022",
      "dependencies": []
    },
    {
      "id": 2,
      "query_str": "What is Uber's revenue for June 2022?",
      "tool_name": "june_2022",
      "dependencies": []
    },
    {
      "id": 3,
      "query_str": "What is Uber's revenue for September 2022?",
      "tool_name": "sept_2022",
      "dependencies": []
    },
    {
      "id": 4,
      "query_str": "Calculate the revenue growth from March to June",
      "tool_name": "revenue_growth_calculator",
      "dependencies": [1, 2]
    },
    {
      "id": 5,
      "query_str": "Calculate the revenue growth from June to September",
      "tool_name": "revenue_growth_calculator",
      "dependencies": [2, 3]
    },
    {
      "id": 6,
      "que

In [19]:
print(str(response))

The revenue growth for Uber over the three quarters in 2022 was $1.489 billion.


RAG Query Planning, bilgi arama ve metin üretme süreçlerini birleştirerek daha doğru, akıcı ve bilgili yanıtlar üretmeyi hedefler. Kullanım alanları şunları içerebilir:

Soru-cevap sistemleri: Daha kapsamlı ve bilgili yanıtlar vermek için.
Özet oluşturma: Bilgileri daha akıcı ve doğru bir şekilde özetlemek için.
Metin oluşturma: Bilgi açısından zengin ve hedef odaklı metinler oluşturmak için.
Diyalog sistemleri: Daha doğal ve bilgili sohbetler gerçekleştirmek için.